In [1]:
from collections import Counter

def extract_ngrams(text, n):
    """Returns a list of n-grams from the input string."""
    tokens = text.strip().split()
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


In [4]:
print(extract_ngrams("the cat sat on the mat", 2))
# → [('the', 'cat'), ('cat', 'sat'), ('sat', 'on'), ('on', 'the'), ('the', 'mat')]


[('the', 'cat'), ('cat', 'sat'), ('sat', 'on'), ('on', 'the'), ('the', 'mat')]


In [11]:
def modified_precision(candidate, reference, n):
    """Compute clipped precision for n-grams."""
    cand_ngrams = Counter(extract_ngrams(candidate, n))
    ref_ngrams = Counter(extract_ngrams(reference, n))

    overlap = {
        ngram: min(count, ref_ngrams[ngram])
        for ngram, count in cand_ngrams.items()
    }
    print(f"🔍 Overlap for n={n} → {overlap}")  # 👈 added line

    clipped_count = sum(overlap.values())
    total_count = sum(cand_ngrams.values())

    if total_count == 0:
        return 0.0
    return clipped_count / total_count


In [12]:
c = "the cat sat on the mat"
r = "the cat lay on the mat"
print(modified_precision(c, r, 3))  # BLEU-2 component


🔍 Overlap for n=3 → {('the', 'cat', 'sat'): 0, ('cat', 'sat', 'on'): 0, ('sat', 'on', 'the'): 0, ('on', 'the', 'mat'): 1}
0.25


In [13]:
import math

def compute_bleu(candidate, reference):
    precisions = []
    for n in range(1, 5):
        p_n = modified_precision(candidate, reference, n)
        precisions.append(p_n)

    # Avoid log(0)
    if any(p == 0 for p in precisions):
        geo_mean = 0
    else:
        logs = [math.log(p) for p in precisions]
        geo_mean = math.exp(sum(logs) / 4)

    # Brevity penalty
    c_len = len(candidate.strip().split())
    r_len = len(reference.strip().split())

    if c_len > r_len:
        bp = 1
    elif c_len == 0:
        bp = 0
    else:
        bp = math.exp(1 - r_len / c_len)

    bleu = bp * geo_mean
    return bleu


In [17]:
cand = "the cat jump on the mat"
ref = "the cat sat on the mat"
print("Final BLEU score:", compute_bleu(cand, ref))


🔍 Overlap for n=1 → {('the',): 2, ('cat',): 1, ('jump',): 0, ('on',): 1, ('mat',): 1}
🔍 Overlap for n=2 → {('the', 'cat'): 1, ('cat', 'jump'): 0, ('jump', 'on'): 0, ('on', 'the'): 1, ('the', 'mat'): 1}
🔍 Overlap for n=3 → {('the', 'cat', 'jump'): 0, ('cat', 'jump', 'on'): 0, ('jump', 'on', 'the'): 0, ('on', 'the', 'mat'): 1}
🔍 Overlap for n=4 → {('the', 'cat', 'jump', 'on'): 0, ('cat', 'jump', 'on', 'the'): 0, ('jump', 'on', 'the', 'mat'): 0}
Final BLEU score: 0.0
